In [26]:
import mlflow
from model.utils.data_preprocess import preprocess_data, split_data
from model.models.models import XGBModel
import pandas as pd 
import json
from mlflow.models import infer_signature


In [21]:
mlflow.login()


INFO:databricks.sdk:loading DEFAULT profile from ~/.databrickscfg: host, username, password
2024/10/05 11:59:12 INFO mlflow.utils.credentials: Successfully connected to MLflow hosted tracking server! Host: https://community.cloud.databricks.com.


In [22]:
mlflow.set_tracking_uri("databricks")


In [23]:
df = preprocess_data(pd.read_csv('/home/miza/housepricesml/model/data/train.csv'))
X_train, X_test, y_train, y_test = split_data(df)

INFO:root:Uncorrelated columns: ['Id', 'BsmtFinSF2', 'LowQualFinSF', 'BsmtHalfBath', '3SsnPorch', 'MiscVal', 'MoSold', 'YrSold']
INFO:root:Insignificant columns: ['Alley', 'LandContour', 'Utilities', 'LandSlope', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'Electrical', 'Functional', 'GarageType', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature']
INFO:root:Missing values columns: [('LotFrontage', False), ('Alley', False), ('MasVnrType', False), ('MasVnrArea', False), ('BsmtQual', False), ('BsmtCond', False), ('BsmtExposure', False), ('BsmtFinType1', False), ('BsmtFinType2', False), ('Electrical', False), ('FireplaceQu', False), ('GarageType', False), ('GarageYrBlt', False), ('GarageFinish', False), ('GarageQual', False), ('GarageCond', False), ('PoolQC', False), ('Fence', False), ('MiscFeature', False)]


In [25]:
with open('/home/miza/housepricesml/best_params.json', "r") as f:
    params = json.load(f)
model = XGBModel(**params)
model.fit(X_train, y_train)
r2 = model.evaluate(X_test, y_test)


In [ ]:
mlflow.set_experiment("houseprices")
with mlflow.start_run():
    mlflow.log_param(params)
    mlflow.log_metric("r2", r2)
    mlflow.set_tag("Training info", "XGB regressor for house prices")
    mlflow.log_artifact("model/data/train.csv")
    
    signature = infer_signature(X_train, model.predict(X_train))
